In [388]:
import pandas as pd
import numpy as np
from transformers import AutoProcessor, CLIPModel
from PIL import Image
import requests
import torch
import time

In [390]:
df = pd.read_csv("20250322_Airline_Reviews_withImageUrls.csv")
df

,Unnamed: 0,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
1,1,AB Aviation,1,"""Not a good airline""",25th June 2019,True,Moroni to Anjouan. It is a very small airline...,NaN,E120,Solo Leisure,...,Moroni to Anjouan,June 2019,2.0,2.0,1.0,1.0,NaN,NaN,2.0,no
2,2,AB Aviation,1,"""flight was fortunately short""",25th June 2019,True,Anjouan to Dzaoudzi. A very small airline an...,NaN,Embraer E120,Solo Leisure,...,Anjouan to Dzaoudzi,June 2019,2.0,1.0,1.0,1.0,NaN,NaN,2.0,no
3,3,Adria Airways,1,"""I will never fly again with Adria""",28th September 2019,False,Please do a favor yourself and do not fly wi...,NaN,NaN,Solo Leisure,...,Frankfurt to Pristina,September 2019,1.0,1.0,NaN,1.0,NaN,NaN,1.0,no
4,4,Adria Airways,1,"""it ruined our last days of holidays""",24th September 2019,True,Do not book a flight with this airline! My fr...,NaN,NaN,Couple Leisure,...,Sofia to Amsterdam via Ljubljana,September 2019,1.0,1.0,1.0,1.0,1.0,1.0,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24349,24349,ZIPAIR,1,"""customer service is terrible""",5th July 2022,False,Bangkok to Tokyo. I’ve flown many low cost ai...,NaN,NaN,Couple Leisure,...,Bangkok to Tokyo,June 2022,2.0,1.0,NaN,1.0,1.0,1.0,1.0,no
24350,24350,ZIPAIR,1,"""Avoid at all costs""",1st June 2022,True,Avoid at all costs. I booked flights to go f...,NaN,NaN,Solo Leisure,...,Singapore to Tokyo,June 2022,NaN,NaN,NaN,NaN,NaN,NaN,1.0,no
24351,24351,ZIPAIR,3,"""Will not recommend to anyone""",31st May 2022,True,Flight was leaving at 23.15 and after an hou...,NaN,NaN,Business,...,Bangkok to Tokyo,May 2022,2.0,4.0,2.0,1.0,NaN,NaN,2.0,no
24352,24352,ZIPAIR,6,"""It was immaculately clean""",23rd May 2022,True,Zipair is JAL’s budget airline. They don’t ha...,NaN,Dreamliner,Business,...,Tokyo to Los Angeles,May 2022,3.0,4.0,3.0,1.0,2.0,5.0,5.0,yes


In [392]:
df.rename(columns={"Unnamed: 0": "RowId"}, inplace=True)

In [394]:
df_images = df[df['Top Review Image Url'].notnull()]
df_images

,RowId,Airline Name,Overall_Rating,Review_Title,Review Date,Verified,Review,Top Review Image Url,Aircraft,Type Of Traveller,...,Route,Date Flown,Seat Comfort,Cabin Staff Service,Food & Beverages,Ground Service,Inflight Entertainment,Wifi & Connectivity,Value For Money,Recommended
0,0,AB Aviation,9,"""pretty decent airline""",11th November 2019,True,Moroni to Moheli. Turned out to be a pretty ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,Moroni to Moheli,November 2019,4.0,5.0,4.0,4.0,NaN,NaN,3.0,yes
13,13,Adria Airways,2,"""Would not fly again""",30th June 2018,True,Ljubljana to Munich. Adria's checkin system ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Business,...,Ljubljana to Munich,June 2018,1.0,2.0,2.0,2.0,NaN,NaN,1.0,no
108,108,Aegean Airlines,5,“claims to be pet-friendly”,27th October 2024,True,While my journey with Aegean Airlines started...,https://www.airlinequality.com/wp-content/uplo...,A320,Business,...,"London Heathrow to Athens, Eleftherios Venizelos",October 2024,2.0,4.0,4.0,1.0,NaN,1.0,1.0,no
109,109,Aegean Airlines,9,“Great value for money!”,13th October 2024,True,Upgraded to business with Plusgrade. Great va...,https://www.airlinequality.com/wp-content/uplo...,A321neo,Business,...,Frankfurt to Paros via Athens,October 2024,5.0,5.0,5.0,4.0,4.0,4.0,3.0,yes
110,110,Aegean Airlines,1,"""75 euro for 1 10 kg bag round trip""",4th October 2024,True,From the bag check showing up 45 min late (...,https://www.airlinequality.com/wp-content/uplo...,NaN,Couple Leisure,...,Rome to Athens,October 2024,1.0,1.0,1.0,1.0,NaN,NaN,1.0,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24298,24298,ZIPAIR,1,"""being periodically harassed and ridiculed""",19th September 2023,True,My review lays with being periodically haras...,https://www.airlinequality.com/wp-content/uplo...,NaN,Solo Leisure,...,Tokyo Narita to San Francisco,September 2023,1.0,1.0,3.0,1.0,1.0,1.0,1.0,no
24304,24304,ZIPAIR,1,"""can highly recommend ZIPAIR""",23rd July 2023,True,A great experience! Everything was easy. Trav...,https://www.airlinequality.com/wp-content/uplo...,NaN,Family Leisure,...,Narita to Seoul,July 2023,5.0,5.0,NaN,5.0,5.0,5.0,5.0,yes
24312,24312,ZIPAIR,1,"""I heard back 3 months later""",18th May 2023,True,ZIPAIR completely broke my brand new luggage ...,https://www.airlinequality.com/wp-content/uplo...,NaN,Couple Leisure,...,San Jose to Tokyo,March 2023,2.0,3.0,3.0,2.0,1.0,1.0,2.0,no
24320,24320,ZIPAIR,1,ZIPAIR customer review,21st April 2023,True,The seats are abysmal for a long-haul flight...,https://www.airlinequality.com/wp-content/uplo...,Boeing 787,Couple Leisure,...,Los Angeles to Tokyo,April 2023,1.0,2.0,1.0,1.0,NaN,1.0,1.0,no


#### CLIP Embed Images

In [397]:
def clip_embedding_image(url):
    try:
        image = Image.open(requests.get(url, stream=True).raw)
        inputs = processor(images=image, return_tensors="pt")
        
        with torch.no_grad():  # Disable gradient computation for inference
            image_embeddings = model.get_image_features(**inputs)
        
        # Normalize the embeddings
        image_embeddings = image_embeddings / image_embeddings.norm(dim=-1, keepdim=True)
    
        #4-second delay
        time.sleep(4)
        
        return image_embeddings.squeeze().tolist()  # Convert tensor to list for storage
        #return image_embeddings
    except Exception as e:
        print(f"Error processing image {image_url}: {e}")
        return None

In [399]:
#clip_embedding_image('https://www.airlinequality.com/wp-content/uploads/2024/10/IMG_5873-500x500.jpeg')

In [401]:
df_images_to_embed = df_images.head(10).copy()
#df_images_to_embed

In [403]:
# Restore from csv backup
df_image_clip_preembed = pd.read_csv("20250322_Airline_Reviews_ImageEmbeddings.csv")
#df_image_clip_preembed

In [405]:
df_images_to_embed['RowId'] = df_images_to_embed['RowId'].astype(int)
df_image_clip_preembed['RowId'] = df_image_clip_preembed['RowId'].astype(int)

In [466]:
if df_image_clip_preembed is None:
    df_images_to_embed['img_clip_embedding'] = df_images_to_embed['Top Review Image Url'].apply(clip_embedding_image)
else:
    #df_merged = df_images_to_embed[['RowId']]\
    #    .merge(df_image_clip_preembed[['RowId','img_clip_embedding']], on='RowId', how='left')
    
    #df_images_to_embed['img_clip_embedding'] = df_merged['img_clip_embedding']
    df_images_to_embed = df_image_clip_preembed

In [409]:
#df_merged

In [464]:
df_images_to_embed['img_clip_embedding']

0    [0.03591400757431984, -0.02889106422662735, 0....
1    [-0.009348137304186821, 0.01193259283900261, 0...
2    [-0.01771741360425949, -0.04664739966392517, -...
3    [0.016391979530453682, 0.025848159566521645, 0...
4    [0.007952975109219551, 0.012641179375350475, -...
5    [-0.016711559146642685, 0.02463185414671898, 0...
6    [-0.010411087423563004, 0.014716237783432007, ...
7    [0.006775944959372282, 0.008402742445468903, 0...
8    [-0.0015220813220366836, 0.021020453423261642,...
9    [0.00717348325997591, 0.023065710440278053, -0...
Name: img_clip_embedding, dtype: object

#### Backup image embeddings to a .csv or .npz file

In [373]:
df_images_to_embed[['RowId','Airline Name','Top Review Image Url','img_clip_embedding']].to_csv("20250322_Airline_Reviews_ImageEmbeddings.csv")

#### Embed text and compute cosine_similarity

In [414]:
def clip_embedding_text(text):
    try:
        inputs = processor(text=[text], return_tensors="pt", padding=True)
        
        with torch.no_grad():
            text_embeddings = model.get_text_features(**inputs)
            
        text_embeddings = text_embeddings / text_embeddings.norm(dim=-1, keepdim=True)
        
        return text_embeddings
    except Exception as e:
        print(f"Error processing text '{text}': {e}")
        return None

In [486]:
import ast

# Compare image embeddings with the text embedding
def compute_similarity(image_embedding, text_embedding):
    if not isinstance(image_embedding, torch.Tensor):  # Convert list to tensor if necessary
        if isinstance(image_embedding, str):
            image_embedding = ast.literal_eval(image_embedding)  # Convert string to list
            image_embedding = torch.tensor(image_embedding)
        
    if image_embedding is not None and text_embedding is not None:
        similarity = torch.nn.functional.cosine_similarity(image_embedding, text_embedding)
        return similarity.item()  # Convert tensor to a scalar
    return None

In [488]:
df_images_to_embed['img_clip_embedding']

0    [0.03591400757431984, -0.02889106422662735, 0....
1    [-0.009348137304186821, 0.01193259283900261, 0...
2    [-0.01771741360425949, -0.04664739966392517, -...
3    [0.016391979530453682, 0.025848159566521645, 0...
4    [0.007952975109219551, 0.012641179375350475, -...
5    [-0.016711559146642685, 0.02463185414671898, 0...
6    [-0.010411087423563004, 0.014716237783432007, ...
7    [0.006775944959372282, 0.008402742445468903, 0...
8    [-0.0015220813220366836, 0.021020453423261642,...
9    [0.00717348325997591, 0.023065710440278053, -0...
Name: img_clip_embedding, dtype: object

In [490]:
text = "a photo of plane"
text_embedding = clip_embedding_text(text)
#text_embedding

In [492]:
# Compute similarity scores with text
df_images_to_embed['img_clip_embedding'] = df_images_to_embed['img_clip_embedding'].to_list()
df_images_to_embed['similarity_with_text'] = df_images_to_embed['img_clip_embedding'].apply(
    lambda img_embed: compute_similarity(img_embed, text_embedding)
)

In [494]:
print(df_images_to_embed[['RowId','Airline Name','Top Review Image Url','similarity_with_text']]\
    .sort_values(by='similarity_with_text', ascending=False).values)

[[136 'Aegean Airlines'
  'https://www.airlinequality.com/wp-content/uploads/2024/03/IMG_1475-500x500.jpeg'
  0.2774202227592468]
 [0 'AB Aviation'
  'https://www.airlinequality.com/wp-content/uploads/2019/11/20191102_123305-500x500.jpg'
  0.275377094745636]
 [13 'Adria Airways'
  'https://www.airlinequality.com/wp-content/uploads/2018/06/IMG_1571-1-500x500.jpg'
  0.26073208451271057]
 [130 'Aegean Airlines'
  'https://www.airlinequality.com/wp-content/uploads/2024/05/IMG_20240518_103907_edit_2247906755096047-500x500.jpg'
  0.2543959319591522]
 [154 'Aegean Airlines'
  'https://www.airlinequality.com/wp-content/uploads/2023/09/16956630467416937441037722578767-500x500.jpg'
  0.24648118019104004]
 [110 'Aegean Airlines'
  'https://www.airlinequality.com/wp-content/uploads/2024/10/PXL_20241004_073123468-500x500.jpg'
  0.22765082120895386]
 [109 'Aegean Airlines'
  'https://www.airlinequality.com/wp-content/uploads/2024/10/20241008_FRA-ATH-A3831_espresso-500x500.jpg'
  0.22450266778469086]